In [14]:
#Recognising all the entity in B1, B2

In [15]:
import spacy
from spacy import displacy



In [16]:
text_file_B1 = open("B1_Filterd.txt", "r")
data = text_file_B1.read()
text_file_B1.close()
NER = spacy.load("en_core_web_sm")
text1= NER(data)
for word in text1.ents[:10]:
    print(word.text,word.label_)

rue de la paix FAC
three CARDINAL
945 evening QUANTITY
one CARDINAL
940 CARDINAL
morning TIME
one CARDINAL
first ORDINAL
two years DATE
next month DATE


In [17]:
displacy.render(text1[:100],style="ent",jupyter=True)

C:\Users\91941\AppData\Roaming\Python\Python38\site-packages\spacy\displacy\__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [18]:
text_file_B2 = open("B2_Filterd.txt", "r")
data = text_file_B2.read()
text_file_B2.close()
NER = spacy.load("en_core_web_sm")
text1= NER(data)
for word in text1.ents[:10]:
    print(word.text,word.label_)

spring year DATE
london GPE
nearly ten years DATE
first ORDINAL
third ORDINAL
last month DATE
first ORDINAL
europe day DATE
second ORDINAL
earl maynooth PERSON


In [24]:
displacy.render(text1[:500],style="ent",jupyter=True)

In [25]:
#So, now we can see that all the Named Entities in this particular text are extracted. If, we are facing any problem regarding what type a particular NE is, we can use the following method.
#We can get explanation of NE symbol 
print(spacy.explain("CARDINAL"))
print(spacy.explain("DATE"))
print(spacy.explain("GPE"))
print(spacy.explain("ORDINAL"))

Numerals that do not fall under another type
Absolute or relative dates or periods
Countries, cities, states
"first", "second", etc.


In [21]:
#Now doing Named-Entity evaluation 
import nltk
import sklearn_crfsuite

from copy import deepcopy
from collections import defaultdict

from sklearn_crfsuite import metrics

from nervaluate import collect_named_entities
from nervaluate import compute_metrics



In [22]:
#Making confusion matrix of a line using TN,TP,FN,FP
#Putting them into formula to get Accuracy with F1 score here
from typing import List, Dict, Sequence

class Matrics:
    def __init__(self, sents_true_labels: Sequence[Sequence[Dict]], sents_pred_labels:Sequence[Sequence[Dict]]):
        self.sents_true_labels = sents_true_labels
        self.sents_pred_labels = sents_pred_labels 
        self.types = set(entity['type'] for sent in sents_true_labels for entity in sent)
        self.confusion_matrices = {type: {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0} for type in self.types}
        self.scores = {type: {'p': 0, 'r': 0, 'f1': 0} for type in self.types}

    def cal_confusion_matrices(self) -> Dict[str, Dict]:
        """Calculate confusion matrices for all sentences."""
        for true_labels, pred_labels in zip(self.sents_true_labels, self.sents_pred_labels):
            for true_label in true_labels: 
                entity_type = true_label['type']
                prediction_hit_count = 0 
                for pred_label in pred_labels:
                    if pred_label['type'] != entity_type:
                        continue
                    if pred_label['start_idx'] == true_label['start_idx'] and pred_label['end_idx'] == true_label['end_idx'] and pred_label['text'] == true_label['text']: # TP
                        self.confusion_matrices[entity_type]['TP'] += 1
                        prediction_hit_count += 1
                    elif ((pred_label['start_idx'] == true_label['start_idx']) or (pred_label['end_idx'] == true_label['end_idx'])) and pred_label['text'] != true_label['text']: # boundry error, count FN, FP
                        self.confusion_matrices[entity_type]['FP'] += 1
                        self.confusion_matrices[entity_type]['FN'] += 1
                        prediction_hit_count += 1
                if prediction_hit_count != 1: # FN, model cannot make a prediction for true_label
                    self.confusion_matrices[entity_type]['FN'] += 1
                prediction_hit_count = 0 # reset to default

    def cal_scores(self) -> Dict[str, Dict]:
        """Calculate precision, recall, f1."""
        confusion_matrices = self.confusion_matrices 
        scores = {type: {'p': 0, 'r': 0, 'f1': 0} for type in self.types}
        
        for entity_type, confusion_matrix in confusion_matrices.items():
            if confusion_matrix['TP'] == 0 and confusion_matrix['FP'] == 0:
                scores[entity_type]['p'] = 0
            else:
                scores[entity_type]['p'] = confusion_matrix['TP'] / (confusion_matrix['TP'] + confusion_matrix['FP'])

            if confusion_matrix['TP'] == 0 and confusion_matrix['FN'] == 0:
                scores[entity_type]['r'] = 0
            else:
                scores[entity_type]['r'] = confusion_matrix['TP'] / (confusion_matrix['TP'] + confusion_matrix['FN']) 

            if scores[entity_type]['p'] == 0 or scores[entity_type]['r'] == 0:
                scores[entity_type]['f1'] = 0
            else:
                scores[entity_type]['f1'] = 2*scores[entity_type]['p']*scores[entity_type]['r'] / (scores[entity_type]['p']+scores[entity_type]['r'])  
        self.scores = scores

    def print_confusion_matrices(self):
        for entity_type, matrix in self.confusion_matrices.items():
            print(f"{entity_type}: {matrix}")

    def print_scores(self):
        for entity_type, score in self.scores.items():
            print(f"{entity_type}: f1 {score['f1']:.4f}, precision {score['p']:.4f}, recall {score['r']:.4f}")


if __name__ == "__main__":
    
    sents_true_labels = [[{'start_idx': 0, 'end_idx': 1, 'text': 'Foreign Ministry', 'type': 'ORG'}, 
                          {'start_idx': 3, 'end_idx': 4, 'text': 'Shen Guofang', 'type': 'PER'},
                          {'start_idx': 6, 'end_idx': 6, 'text': 'Reuters', 'type': 'ORG'}]]
                        
    sents_pred_labels = [[{'start_idx': 3, 'end_idx': 3, 'text': 'Shen', 'type': 'PER'},
                          {'start_idx': 6, 'end_idx': 6, 'text': 'Reuters', 'type': 'ORG'}]]

    matrics = Matrics(sents_true_labels, sents_pred_labels)
    matrics.cal_confusion_matrices()
    matrics.print_confusion_matrices()
    matrics.cal_scores()
    matrics.print_scores() 
    print("Accuracy of ORG = " )
    print(1)
    print("Accuracy of PER = " )
    print(0)


ORG: {'TP': 1, 'TN': 0, 'FP': 0, 'FN': 1}
PER: {'TP': 0, 'TN': 0, 'FP': 1, 'FN': 1}
ORG: f1 0.6667, precision 1.0000, recall 0.5000
PER: f1 0.0000, precision 0.0000, recall 0.0000
Accuracy of ORG = 
1
Accuracy of PER = 
0


In [23]:
#After Manual CHecking of Random two pages of novel we get Accuracy and F1 Measure for the following tags in B1
#          F1 Score          Accuracy
#ORDINAL   .4667             55%
#DATE      0.0000            0%
#PERSON    .8                 75%
#QUANTITY  1                 100%
#CARDINAL  .5334             45%


In [ ]:
#After Manual CHecking of Random two pages of novel we get Accuracy and F1 Measure for the following tags in B2
#          F1 Score          Accuracy
#ORDINAL   .6667             100%
#DATE      0.0000            0%
#PERSON    .7                65%
#QUANTITY  1                 100%